Pockemon data ingestion

In [ ]:
# To connect to the Jupiter Server
# In the docker attach screen, look for a message like this:
#    http://127.0.0.1:8888/lab?token=xxxx
# The "xxxx" is the password to be used when the Jupyter Kernel Connection ask for it...
# Then select the "Existing Jupiter Server" option
# Specify the URL: http://127.0.0.1:8888
# Specify the password: xxxx
# Select the desired Kernel from the list

# References:
# google: what is the default python jupiterlab server password
# https://stackoverflow.com/questions/41117554/what-is-default-password-for-jupyter-created-on-googles-data-proc

# https://github.com/jupyter/notebook/commit/7fa5d5a1be147e9c8e14f61a2f4b3c0db1e2c00b
# For servers with token-authentication enabled, the URL in the above listing will include the token,
# so you can copy and paste that URL into your browser to login.

Step 1: Setup Environment

In [ ]:
# Init Minio
!sh /home/PyCon2024/Project/Scripts/1.init_minio.sh "data"

In [ ]:
# Install necessary packages
import sys
!{sys.executable} -m pip install pyspark
!{sys.executable} -m pip install s3fs
!{sys.executable} -m pip install minio
!{sys.executable} -m pip install pyhive
!{sys.executable} -m pip install trino

In [ ]:
# Install dotenv to load environment variables
!{sys.executable} -m pip install python-dotenv

In [ ]:
# Load environment variables
import os
from dotenv import load_dotenv
load_dotenv('minio.env')

# Access the environment variables
minio_access_key = os.getenv('MINIO_ACCESS_KEY')
minio_secret_key = os.getenv('MINIO_SECRET_KEY')
minio_endpoint = os.getenv('MINIO_ENDPOINT', "http://minio:9000")
minio_bucket_name = os.getenv('MINIO_BUCKET_NAME', "data-lakehouse")

In [ ]:
print("Minio Access Key:", minio_access_key)
print("Minio Secret Key:", minio_secret_key)
print("Minio Endpoint:", minio_endpoint)
print("Minio Bucket Name:", minio_bucket_name)

In [ ]:
# Import Spark libraries
import pyspark
import pyspark.sql.functions as sqlF
from pyspark.sql import SparkSession

In [ ]:
# Import SLAlchemy and Pandas libraries
from sqlalchemy.sql import text
from sqlalchemy import create_engine
import pandas as pd

In [ ]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("PokemonDataIngestion") \
    .config("spark.driver.host", "localhost") \
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0,org.apache.hadoop:hadoop-aws:3.3.3") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("hive.metastore.uris", "thrift://metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [ ]:
# Check spark configuration
spark.sparkContext.getConf().getAll()

Step 2: Build the data frames

In [ ]:
df = spark.read.json('s3a://data-lakehouse/Raw/data/pokemon/')

In [ ]:
# df.show(truncate=False, n=10)
df.select('name').show()

In [ ]:
df.printSchema()

In [ ]:
df.select("name", "moves").show(truncate=True)

In [ ]:
df.write.format("delta").save("s3a://data-lakehouse/bronze/pokemons")

In [ ]:
spark.sql("SHOW TABLES from bronze").show()

In [ ]:
additional_options = {"path": "s3a://pruebas/bronze/new_table4", "overwriteSchema": "true", "mergeschema": "true"}
df.write.format("delta").options(**additional_options).mode("overwrite").saveAsTable("bronze.tabla4")

In [ ]:
df_2 = spark.read.format("delta").table("bronze.pokemons")

In [ ]:
df_2.select("name").show()

In [ ]:
spark.sql("DESCRIBE HISTORY bronze.pokemons").show()